# receive_message

> Receiving a CAN message from a bus
> using the python-can library
> testing on vcan interface

In [ ]:
#| default_exp receive_message

In [ ]:
#| hide
from __future__ import annotations
from nbdev.showdoc import *
from fastcore.test import *


In [ ]:
from pprint import pprint
import subprocess

In [ ]:
#| export
import os
import signal
import time
import sys
import io
from multiprocessing import Event
from multiprocessing import synchronize, Manager
from multiprocessing.managers import DictProxy
from typing import Optional
from datetime import datetime
import json
import argparse

In [ ]:
#| export
import can
# from can.interface import Bus
# from can import Message 
import cantools
from cantools.database import Message as MessageTpl
from cantools.database.can.database import Database

In [ ]:
#| export
def get_argparser() -> argparse.ArgumentParser:
    """_summary_ get CAN bus, dbc config and the message to send

    Returns:
        argparse.ArgumentParser: _description_
    """

    parser = argparse.ArgumentParser("Get the CAN Bus channel, bitrate and dbc path")

    parser.add_argument(
        "-t",
        "--type",
        type=str,
        default="socketcan",
        help="The type of the CAN bus",
    )

    parser.add_argument(
        "-c",
        "--channel",
        type=str,
        default="vcan1",
        help="The CAN bus channel to connect to",
    )

    parser.add_argument(
        "-b", "--bitrate", type=int, default=250000, help="The bitrate of the CAN bus"
    )

    parser.add_argument(
        "-d",
        "--dbc",
        type=str,
        default="../res/motohawk_new.dbc",
        help="The path to the dbc file",
    )

    parser.add_argument(
        "-m",
        "--message",
        type=str,
        default="ExampleMessage",
        help="The message to send",
    )

    parser.add_argument(
        "-e",
        "--extended",
        action="store_true",
        help="If the arbitration id is extended",
    )

    return parser


In [ ]:
db = cantools.database.load_file('../res/motohawk_new.dbc')
db.messages
example_message: MessageTpl = db.get_message_by_name('ExampleMessage')
pprint(example_message.signals)
# pprint(example_message.__dict__)
example_message.frame_id

[message('ExampleMessage', 0x1f0, False, 8, {None: 'Example message used as template in MotoHawk models.'}),
 message('NewMessage', 0x254, False, 8, {None: 'self made message'})]

[signal('Enable', 7, 1, 'big_endian', False, None, 1, 0, None, None, '-', False, None, {0: 'Disabled', 1: 'Enabled'}, None, None),
 signal('AverageRadius', 6, 6, 'big_endian', False, None, 0.1, 0, 0, 5, 'm', False, None, None, None, None),
 signal('Temperature', 0, 12, 'big_endian', True, None, 0.01, 250, 229.52, 270.47, 'degK', False, None, None, None, None)]


496

In [ ]:
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe vcan
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev vcan0 type vcan
# sshpass -v -p asdf sudo ip link add dev vcan0 type vcan
!ip link show vcan0
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 type vcan bitrate 500000  # vcan Does not SUPPORT set bitrate on command line!
# !sshpass -p asdf sudo ip link add dev vcan0 type vcan
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up vcan0
# !sshpass -v -p asdf sudo ip link set up vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.


SSHPASS: read: 

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 

SSHPASS: read: RTNETLINK answers: File exists

42: vcan0: <NOARP,UP,LOWER_UP> mtu 72 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/can 
SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



In [ ]:
data_dict ={'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}
data_json_bytes = json.dumps(data_dict).encode('utf-8')
data_json_bytes
json.loads(data_json_bytes.decode())

can_data = example_message.encode({'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1})
example_message.decode(can_data)

b'{"Temperature": 250.1, "AverageRadius": 3.2, "Enable": 1}'

{'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}

{'Enable': 'Enabled', 'AverageRadius': 3.2, 'Temperature': 250.1}

In [ ]:
f = io.BytesIO(data_json_bytes)

In [ ]:
bus = can.interface.Bus(bustype='socketcan', channel='vcan0', bitrate=250000)
data = example_message.encode({'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1})
message_to_send = can.Message(arbitration_id=example_message.frame_id, data=data, is_extended_id=False) 	
# can_bus.send(message)

In [ ]:
manager = Manager()
message_proxy = manager.dict()

In [ ]:
#| export
def receive_msg(db:Database, message:str, channel:str, bitrate:int, bus_type: str) -> dict:

    print('Receiving message')
    with can.interface.Bus(bustype=bus_type, channel=channel, bitrate=bitrate) as bus:
        msg:Message = bus.recv()
    print('Received message')

    return {'timestamp': datetime.fromtimestamp(msg.timestamp),
            'payload': db.decode_message(msg.arbitration_id,msg.data)
            }


In [ ]:
#| hide 
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
def receive_message(message_proxy: DictProxy,bus: can.interface.Bus)->None:
	print('waiting for message')
	msg:Message = bus.recv()
	print('message received')
	message_proxy['timestamp'] = msg.timestamp
	message_proxy['arbitration_id'] = msg.arbitration_id
	message_proxy['data']=msg.data

In [ ]:

os.getcwd()

'/home/n/devel/anting_remote/eos/tspace/candycan/nbs'

In [ ]:
# proc = subprocess.Popen(['python', 'message_send.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
# print(f'PARENT: {proc.pid} before signaling child')
# sys.stdout.flush()
# time.sleep(1)


In [ ]:

# receive_message(message_proxy, bus)

In [ ]:
# print(f'PARENT: {proc.pid} signaling child')
# sys.stdout.flush()
# os.kill(proc.pid, signal.SIGUSR1)
# stdout_value = proc.communicate()[0].decode('utf-8')
# print(f'stdout: {repr(stdout_value)}')

In [ ]:
# datetime.fromtimestamp(message_proxy['timestamp']),db.decode_message(message_proxy['arbitration_id'],message_proxy['data'])

In [ ]:
# close and remove vcan0
# !sshpass -v -p  asdf sudo ip link delete vcan0 

!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set down vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



In [ ]:

!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link delete vcan0 

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



In [ ]:
#| export
if __name__ == "__main__" and "__file__" in globals():   # in order to be compatible for both script and notebnook

    # print(os.getcwd())
    p = get_argparser()
    args = p.parse_args()

    try:
        db: Database = cantools.database.load_file(args.dbc)
    except FileNotFoundError as e:
        print(f"File not found: {e}")


    
    msg = receive_msg(db=db,
                    message=args.message,
                    channel=args.channel,
                    bitrate=args.bitrate,
                    bus_type=args.type
                    )
    print(msg)
    # sys.stdout.flush()



In [ ]:
#| hide 
import nbdev; nbdev.nbdev_export()